In [7]:
import gym
import pylab
import random
import numpy as np
from collections import deque
import tflearn
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import sgd


gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)

## Task: fill empty spaces in the following agent code

In [21]:
class DeepQAgent:
    def __init__(self, state_size, action_size, render=True):
        # Tip: if you are training this on AWS the best way is to turn off rendering
        # and load it later with the serialized model
        self.render = render
        self.state_size = state_size
        self.action_size = action_size

        self.discount_factor = 0.99
        self.learning_rate = 0.1
        self.epsilon = 1.0
        self.epsilon_min = 0.005
        self.epsilon_decay = (self.epsilon - self.epsilon_min) / 50000
        self.batch_size = 64
        self.train_start = 1000
        # replay memory
        self.memory = deque(maxlen=10000)

        self.model = self.build_model()
        self.target_model = self.build_model()
        self.update_target_model()

    def build_model(self):
        model = Sequential()
        model.add(Dense(10, activation='relu', input_dim = self.state_size))
        model.add(Dense(10, activation='relu'))
        model.add(Dense(self.action_size))
        
        model.compile(sgd(lr=self.learning_rate), "mse")
        return model

    def update_target_model(self):
        print("update")
        """Update your target model to the model you are currently learning at regular time intervals"""        
        self.target_model.set_weights(self.model.get_weights())


    def get_action(self, state):
        """The choice of action uses the epsilon-greedy policy for the current network."""
        if np.random.rand() <= self.epsilon:
            #if (state[0][0] < -0.6 and state[0][1] < -0.01):
            #    return 0
            #elif (state[0][0] > -0.4 and state[0][1] > 0.01):
            #    return 1
            #elif (state[0][0] < -0.6 and state[0][1] > -0.01):
            #    return 1
            #elif (state[0][0] > -0.4 and state[0][1] < 0.01):
            #    return 0
            #elif (state[0][0] > -0.6 and state[0][0] < -0.4 and state[0][1] < 0):
            #    return 0
            #elif (state[0][0] > -0.6 and state[0][0] < -0.4 and state[0][1] > 0):
            #    return 1
            
            return random.randrange(self.action_size)
        else:
            q_value = self.model.predict(state)
            return np.argmax(q_value[0])

    def replay_memory(self, state, action, reward, next_state, done):
        """Save <s, a, r, s'> to replay_memory"""
        if action == 2:
            action = 1
        self.memory.append((state, action, reward, next_state, done))
        if self.epsilon > self.epsilon_min:
            self.epsilon -= self.epsilon_decay
            # print(len(self.memory))

    def train_replay(self):
        """Random sampling of batch_size samples from replay memory"""
        if len(self.memory) < self.train_start:
            return
        batch_size = min(self.batch_size, len(self.memory))
        mini_batch = random.sample(self.memory, batch_size)

        update_input = np.zeros((batch_size, self.state_size))
        update_target = np.zeros((batch_size, self.action_size))

        for i in range(batch_size):
            state, action, reward, next_state, done = mini_batch[i]
            target = self.model.predict(state)[0]
            #print(state, action, reward, next_state, done)

            # As in queuing, it gets the maximum Q Value at s'. However, it is imported from the target model.
            if done:
                target[action] = reward
            else:
                target[action] = reward + self.discount_factor * \
                                          np.amax(self.target_model.predict(next_state)[0])
                #print('new target action', reward, self.discount_factor, np.amax(self.target_model.predict(next_state)[0]))
                #print(next_state, np.amax(self.target_model.predict(next_state)[0]))
                
            update_input[i] = state
            update_target[i] = target

        #print('input', update_input)
        #print('target', update_target)
        # You can create a minibatch of the correct target answer and the current value of your own,
        self.model.fit(update_input, update_target, batch_size=batch_size, epochs=1, verbose=0)

    def load_model(self, name):
        self.model.load_model(name)

    def save_model(self, name):
        self.model.save(name)


In [22]:
env = gym.make('MountainCar-v0')
state_size = env.observation_space.shape[0] # should be equal 2
ACTION_SIZE = 2
agent = DeepQAgent(state_size, ACTION_SIZE)
#agent.load_model("./save_model/model")
scores, episodes = [], []
N_EPISODES = 4000

update


In [23]:
for e in range(N_EPISODES):
    done = False
    score = 0
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    print("state", state)

    # Action 0 (left), 1 (do nothing), 3 (declare fake_action to avoid doing nothing
    fake_action = 0

    # Counter for the same action 4 times
    action_count = 0

    while not done:
        #if agent.render:
        #    env.render()

        # Select an action in the current state and proceed to a step
        action_count = action_count + 1

        if action_count == 4:
            action = agent.get_action(state)
            action_count = 0

            if action == 0:
                fake_action = 0
            elif action == 1:
                fake_action = 2

        # Take 1 step with the selected action
        next_state, reward, done, info = env.step(fake_action)
        next_state = np.reshape(next_state, [1, state_size])
        #print('env: ', next_state, reward, done, info)
        # Give a penalty of -100 for actions that end an episode
        # reward = reward if not done else -100

        # Save <s, a, r, s'> to replay memory
        agent.replay_memory(state, fake_action, reward, next_state, done)
        # Continue to learn every time step
        agent.train_replay()
        score += reward
        state = next_state

        if done:
            env.reset()
            # Copy the learning model for each episode to the target model
            agent.update_target_model()

            # For each episode, the time step where cartpole stood is plot
            scores.append(score)
            episodes.append(e)
            print("episode:", e, "  score:", score, "  memory length:", len(agent.memory),
                  "  epsilon:", agent.epsilon)

    # Save model for every 50 episodes
    if e % 50 == 0:
        agent.save_model("./save_model/model")

state [[-0.42803098  0.        ]]
update
episode: 0   score: -200.0   memory length: 200   epsilon: 0.9960200000000077
state [[-0.48558155  0.        ]]
update
episode: 1   score: -200.0   memory length: 400   epsilon: 0.9920400000000154
state [[-0.52362841  0.        ]]
update
episode: 2   score: -200.0   memory length: 600   epsilon: 0.988060000000023
state [[-0.51127663  0.        ]]
update
episode: 3   score: -200.0   memory length: 800   epsilon: 0.9840800000000307
state [[-0.4928123  0.       ]]
update
episode: 4   score: -200.0   memory length: 1000   epsilon: 0.9801000000000384
state [[-0.58122748  0.        ]]
update
episode: 5   score: -200.0   memory length: 1200   epsilon: 0.9761200000000461
state [[-0.57579039  0.        ]]
update
episode: 6   score: -200.0   memory length: 1400   epsilon: 0.9721400000000537
state [[-0.43192112  0.        ]]
update
episode: 7   score: -200.0   memory length: 1600   epsilon: 0.9681600000000614
state [[-0.51513039  0.        ]]
update
episod

update
episode: 68   score: -200.0   memory length: 10000   epsilon: 0.7253800000005297
state [[-0.54394262  0.        ]]
update
episode: 69   score: -200.0   memory length: 10000   epsilon: 0.7214000000005374
state [[-0.49639544  0.        ]]
update
episode: 70   score: -200.0   memory length: 10000   epsilon: 0.7174200000005451
state [[-0.56452713  0.        ]]
update
episode: 71   score: -200.0   memory length: 10000   epsilon: 0.7134400000005527
state [[-0.48023568  0.        ]]
update
episode: 72   score: -200.0   memory length: 10000   epsilon: 0.7094600000005604
state [[-0.52009665  0.        ]]
update
episode: 73   score: -200.0   memory length: 10000   epsilon: 0.7054800000005681
state [[-0.47839659  0.        ]]
update
episode: 74   score: -200.0   memory length: 10000   epsilon: 0.7015000000005758
state [[-0.53833185  0.        ]]
update
episode: 75   score: -200.0   memory length: 10000   epsilon: 0.6975200000005835
state [[-0.55118592  0.        ]]
update
episode: 76   sco

update
episode: 136   score: -200.0   memory length: 10000   epsilon: 0.45474000000092546
state [[-0.54082439  0.        ]]
update
episode: 137   score: -200.0   memory length: 10000   epsilon: 0.45076000000092203
state [[-0.53546547  0.        ]]
update
episode: 138   score: -128.0   memory length: 10000   epsilon: 0.44821280000091984
state [[-0.55867813  0.        ]]
update
episode: 139   score: -200.0   memory length: 10000   epsilon: 0.4442328000009164
state [[-0.58292479  0.        ]]
update
episode: 140   score: -200.0   memory length: 10000   epsilon: 0.440252800000913
state [[-0.40926182  0.        ]]
update
episode: 141   score: -200.0   memory length: 10000   epsilon: 0.43627280000090957
state [[-0.5487637  0.       ]]
update
episode: 142   score: -200.0   memory length: 10000   epsilon: 0.43229280000090614
state [[-0.45618541  0.        ]]
update
episode: 143   score: -200.0   memory length: 10000   epsilon: 0.4283128000009027
state [[-0.54887082  0.        ]]
update
episode

update
episode: 203   score: -200.0   memory length: 10000   epsilon: 0.19110480000078073
state [[-0.42772852  0.        ]]
update
episode: 204   score: -200.0   memory length: 10000   epsilon: 0.18712480000078285
state [[-0.4944659  0.       ]]
update
episode: 205   score: -200.0   memory length: 10000   epsilon: 0.18314480000078498
state [[-0.53765654  0.        ]]
update
episode: 206   score: -200.0   memory length: 10000   epsilon: 0.1791648000007871
state [[-0.59688304  0.        ]]
update
episode: 207   score: -200.0   memory length: 10000   epsilon: 0.17518480000078923
state [[-0.59509041  0.        ]]
update
episode: 208   score: -200.0   memory length: 10000   epsilon: 0.17120480000079136
state [[-0.58594207  0.        ]]
update
episode: 209   score: -200.0   memory length: 10000   epsilon: 0.16722480000079348
state [[-0.53616726  0.        ]]
update
episode: 210   score: -200.0   memory length: 10000   epsilon: 0.1632448000007956
state [[-0.42653875  0.        ]]
update
episo

update
episode: 269   score: -191.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.50217368  0.        ]]
update
episode: 270   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.5870195  0.       ]]
update
episode: 271   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.53914996  0.        ]]
update
episode: 272   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.41121877  0.        ]]
update
episode: 273   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.5841355  0.       ]]
update
episode: 274   score: -159.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.51345778  0.        ]]
update
episode: 275   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.54837095  0.        ]]
update
episode: 276   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.57254255  0.        ]]
upda

...

update
episode: 3701   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.42557885  0.        ]]
update
episode: 3702   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.57160504  0.        ]]
update
episode: 3703   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.48391802  0.        ]]
update
episode: 3704   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.42553443  0.        ]]
update
episode: 3705   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.51608504  0.        ]]
update
episode: 3706   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.54703516  0.        ]]
update
episode: 3707   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.53361561  0.        ]]
update
episode: 3708   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.56491833  0.   

update
episode: 3767   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.57218388  0.        ]]
update
episode: 3768   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.4009117  0.       ]]
update
episode: 3769   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.53449622  0.        ]]
update
episode: 3770   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.50112771  0.        ]]
update
episode: 3771   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.56266005  0.        ]]
update
episode: 3772   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.57775483  0.        ]]
update
episode: 3773   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.58988484  0.        ]]
update
episode: 3774   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.53241195  0.     

update
episode: 3833   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.44059191  0.        ]]
update
episode: 3834   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.43077717  0.        ]]
update
episode: 3835   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.4810235  0.       ]]
update
episode: 3836   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.50973347  0.        ]]
update
episode: 3837   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.41536282  0.        ]]
update
episode: 3838   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.40057557  0.        ]]
update
episode: 3839   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.55692238  0.        ]]
update
episode: 3840   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.50469297  0.     

update
episode: 3899   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.57234786  0.        ]]
update
episode: 3900   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.49710014  0.        ]]
update
episode: 3901   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.42393338  0.        ]]
update
episode: 3902   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.41342292  0.        ]]
update
episode: 3903   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.47179323  0.        ]]
update
episode: 3904   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.58130442  0.        ]]
update
episode: 3905   score: -139.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.53202752  0.        ]]
update
episode: 3906   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.5309918  0.    

update
episode: 3965   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.52500986  0.        ]]
update
episode: 3966   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.40789554  0.        ]]
update
episode: 3967   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.52801487  0.        ]]
update
episode: 3968   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.48704744  0.        ]]
update
episode: 3969   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.57119853  0.        ]]
update
episode: 3970   score: -124.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.47541054  0.        ]]
update
episode: 3971   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.44769347  0.        ]]
update
episode: 3972   score: -200.0   memory length: 10000   epsilon: 0.004980100000801017
state [[-0.4290338  0.    